# <div style="padding:20px;color:white;margin:0;font-size:200%;text-align:center;display:fill;border-radius:5px;background-color:#AF7AC5;overflow:hidden;font-weight:500">TPS JUNE 2022
</div>

 ### Most of the public notebooks in this competition has assumed that single type of algorithum (either XGB, Pytorch or LGBM) is applicable for all the columns. I would like to propose a different way of handling where each column can is treated individually and based on the training metric best algorithum is choosen for a given column.
 ### If you are a beginner, see my other notebook for imputation tutorial [notebook](https://www.kaggle.com/code/abdulravoofshaik/quick-eda-and-missing-values-tutorial). 
 ### This notebook assumes that you are familer with EDA and know how to use single algorithum. If you are not, please see these notebooks [one](https://www.kaggle.com/code/dwin183287/tps-june-2022-eda) and [two](https://www.kaggle.com/code/abdulravoofshaik/top-3-solution-lgbm-mean).
 
 ### PS: This notebook is computationally expensive and takes longtime to run.  



<div style="color:white;display:fill;border-radius:8px;
            background-color:#D7BDE2;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>1.0 | Load data</b></p>
</div>

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import missingno as msno
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import matplotlib.colors
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import seaborn as sns
data = pd.read_csv("../input/tabular-playground-series-jun-2022/data.csv")
Target = pd.read_csv("../input/tabular-playground-series-jun-2022/sample_submission.csv", index_col='row-col')

<div style="color:white;display:fill;border-radius:8px;
            background-color:#D7BDE2;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>2.0 | Preprocessing</b></p>
</div>

### Lets start with column F_1_0 and try to replace the NaN values. First we need to find out where are the missing values located in this column. As shown below figure, we need to split our data into two sets. 
### Training set: It consists of known values for F_1_0 column, which means all the rows with non-NaN value in F_1_0 column. 
### Test set: It consists of Unknown values for F_1_0 column, which means all the rows with  NaN value in F_1_0 column.
### Here we apply three different algorithums (LGBM, XGB and Catboost) to train on each column. Based on the error analysis we choose the best model to predict NaN values.
### It has been well established in the previous notebook xxx that there are no missing values for columns starting with F_2. Columns starting with F_4 are correlated with each other. 


In [ ]:
## we apply the same concept to individual column and develop 80 individual models. As we have noticed earlier, dataset has four different subsets. Lets divide them.
# this code snippet is taken from https://www.kaggle.com/code/martynovandrey/tps-jun-22-splitted-dataset-24x-faster. Consider upvoting the original author also
features = list(data.columns)
features_1, features_2, features_3, features_4 = [], [], [], []
F = [[], [], [], [], []]
for feature in features:
    for i in [1, 2, 3, 4]:
        if feature.split('_')[1] == str(i):
            F[i].append(feature)
df = [[], [], [], [], []]

for i in [1, 2, 3, 4]:
    df[i] = data[F[i]]

<div style="color:white;display:fill;border-radius:8px;
            background-color:#D7BDE2;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.0 | Modeling</b></p>
</div>

<div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana;">
    📌 Important note:  The strategy is to use LGBM regression, XGB regression and Catboost Regression and choose the best model for each column for F4. For F1 and F3 I will use mean imputer.
</div>

In [ ]:
%%time
# Ensemble regression
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
import statistics
from sklearn.metrics import r2_score
Ensemble_df = pd.DataFrame()
xgb.set_config(verbosity=0)

kfold = KFold(n_splits = 3, shuffle=True, random_state = 0) # you can increase the n_splits value to 10, to minimize the runtime I have used 3

lgbm_params = {'random_state': 22,
#           'device' : 'gpu',   #uncomment it if you want to use GPU
          'n_estimators': 4000, # you can increase the n_splits value to >10000, to minimize the runtime I have used 4000
          'learning_rate' : 0.1,
          'metric' : 'r2'}

xgb_params = {'random_state': 22,
#           'tree_method' : 'gpu_hist',   #uncomment it if you want to use GPU
          'n_estimators': 20,  # you can increase the n_splits value to >1000, to minimize the runtime I have used 20
          'metric' : 'r2'} 

cat_params = {'random_state': 123,
#           'task_type' : 'GPU',   #uncomment it if you want to use GPU
          'iterations': 500,  # you can increase the n_splits value to >2000, to minimize the runtime I have used 100
         'loss_function': 'RMSE',}


for i in [4]:
    dummy_df = pd.DataFrame()
    LGBM_df = pd.DataFrame()
    XGB_df = pd.DataFrame()
    CAT_df = pd.DataFrame()
    ENSEMBLE_df = pd.DataFrame()
    
    col_train = pd.DataFrame()
    col_test = pd.DataFrame()
    
    dummy_df=df[i].copy()
    LGBM_df=df[i].copy()
    XGB_df = df[i].copy()
    CAT_df= df[i].copy()
    ENSEMBLE_df= df[i].copy()
    
    for column in dummy_df.columns: 
        print('----------------------------------------Processing Colunm Name : ', column)
        lgbm_feature_imp, y_pred_list_lgbm, y_true_list, lgbm_R2_list  = [],[],[],[]
        xgb_feature_imp, y_pred_list_xgb, xgb_R2_list  = [],[],[]
        cat_feature_imp, y_pred_list_cat, cat_R2_list  = [],[],[]
        if dummy_df[column].isnull().sum() == 0:
            print(dummy_df[column].isnull().sum())
            continue    # continue as no NaN values found in this column
        col_nan_ix = dummy_df[dummy_df[column].isnull()].index  # identify the rows which has NaN in column 
        col_train = dummy_df.drop(col_nan_ix, axis = 0)  #training set which has fixed value but other columns might have NaN values
        col_test = dummy_df[dummy_df.index.isin(col_nan_ix)] # test set which has NaN value in the test set
        lgbm_model = LGBMRegressor(**lgbm_params)
        xgb_model = XGBRegressor(**xgb_params)
        cat_model = CatBoostRegressor(**cat_params)
        X = col_train.drop([column],axis=1).reset_index(drop=True)
        y = col_train[column].reset_index(drop=True)
        for fold, (train_index, val_index) in enumerate(kfold.split(X, y)): # split the trainset to train and valid set
            print("==fold==", fold)
            X_train = X.loc[train_index]
            X_val = X.loc[val_index]
            y_train = y.loc[train_index]
            y_val = y.loc[val_index]
            print("Fitting CAT for the column",column)
            cat_model.fit(X_train,y_train,logging_level='Silent')
            print("Fitting LGBM for the column",column)
            lgbm_model.fit(X_train,y_train)
            print("Fitting XGB for the column",column)
            xgb_model.fit(X_train,y_train)

            y_pred_lgbm = lgbm_model.predict(X_val)
            y_pred_xgb = xgb_model.predict(X_val)
            y_pred_cat = cat_model.predict(X_val)
            y_pred_list_lgbm = np.append(y_pred_list_lgbm, y_pred_lgbm)
            y_pred_list_xgb= np.append(y_pred_list_xgb, y_pred_xgb)
            y_pred_list_cat= np.append(y_pred_list_cat, y_pred_cat)
            y_true_list = np.append(y_true_list, y_val)
            lgbm_R2_list.append(r2_score(y_val,y_pred_lgbm)) # Collecting the LGBM R2 score for each fold
            xgb_R2_list.append(r2_score(y_val,y_pred_xgb))   # Collecting the XGB R2 score for each fold
            cat_R2_list.append(r2_score(y_val,y_pred_cat))   # Collecting the CAT R2 score for each fold
        LGBM_df[column][col_nan_ix] = lgbm_model.predict(col_test.drop([column],axis=1))
        XGB_df[column][col_nan_ix] = xgb_model.predict(col_test.drop([column],axis=1))
        CAT_df[column][col_nan_ix] = cat_model.predict(col_test.drop([column],axis=1))
        print("R2_lgbm",statistics.mean(lgbm_R2_list))
        print("XGB_lgbm",statistics.mean(xgb_R2_list))
        print("CAT_lgbm",statistics.mean(cat_R2_list))
        Ensebmle_error = {'LGBM' : statistics.mean(lgbm_R2_list),  # calculating mean R2 of all the folds
        'XGB' : statistics.mean(xgb_R2_list),
        'CAT' : statistics.mean(cat_R2_list)}
        best_model= max(Ensebmle_error, key=Ensebmle_error.get)  # identifying best model by comparing R2 values
        if best_model=='LGBM':
          print("best model is LGBM for the column",column)
          ENSEMBLE_df[column][col_nan_ix] = lgbm_model.predict(col_test.drop([column],axis=1))
        elif best_model=='XGB':
          print("best model is XGB for the column",column)
          ENSEMBLE_df[column][col_nan_ix] = xgb_model.predict(col_test.drop([column],axis=1))
        else:
          print("best model is CAT for the column",column)
          ENSEMBLE_df[column][col_nan_ix] = cat_model.predict(col_test.drop([column],axis=1))
df[4]=ENSEMBLE_df.copy()

### Now we use simple meanimputer for subset-1 and subset-3. Note that subset-2 has no missing values

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(
        missing_values=np.nan,
        strategy='mean') 
for i in [1,3]:    
    df[i][:] = imp.fit_transform(df[i])

In [ ]:
Merged_Subsets = pd.concat([df[1], df[2], df[3], df[4]], axis=1)
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = Merged_Subsets.loc[row, col]

# submission.to_csv('submission.csv')

###  We need to optimize the model by chaning the algorithums and optimzing the hyperparameters. So far, after tuning, my results are as follows. I will update the code after finalizing the model. You can also try.

In [ ]:
submission= pd.read_csv("../input/tps-june2022-bestofthree/submission_best_v4.csv")
submission.to_csv('submission.csv', index=False)

### Work in Progress.....

<img src="https://media1.giphy.com/media/5nvQ7fBWhPVXXOcfRI/giphy.gif?cid=6c09b952a8132b3cf6fd8d9503e91775e810cf6d3d504bc4&rid=giphy.gif&ct=g">

<div style="color:white;display:fill;border-radius:8px;
            background-color:#E59866;font-size:150%;
            font-family:Nexa;letter-spacing:0.5px">
    <p style="padding: 8px;color:black;"><b>3.0 | References</b></p>
</div>

https://towardsdatascience.com/using-the-missingno-python-library-to-identify-and-visualise-missing-data-prior-to-machine-learning-34c8c5b5f009 <br>
https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4 <br>
https://www.kaggle.com/code/parulpandey/a-guide-to-handling-missing-values-in-python/notebook <br>
https://www.kaggle.com/code/residentmario/using-missingno-to-diagnose-data-sparsity/notebook <br>
https://www.analyticsvidhya.com/blog/2021/05/dealing-with-missing-values-in-python-a-complete-guide/ <br>
https://www.kaggle.com/code/calebreigada/getting-started-eda-preprocessing <br>
https://medium.com/swlh/impute-missing-values-the-right-way-c63735fccccd <br>

